In [1]:
import requests
import json
import time
final_data = []
class GooglePlaces(object):
    def __init__(self, apiKey):
        super(GooglePlaces, self).__init__()
        self.apiKey = apiKey

    def search_places_by_coordinate(self, location, radius, types):
        endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        places = []
        params = {
            'location': location,
            'radius': radius,
            'types': types,
            'key': self.apiKey
        }
        res = requests.get(endpoint_url, params = params)
        results =  json.loads(res.content)
        places.extend(results['results'])
        time.sleep(2)
        while "next_page_token" in results:
            params['pagetoken'] = results['next_page_token'],
            res = requests.get(endpoint_url, params = params)
            results = json.loads(res.content)
            places.extend(results['results'])
            time.sleep(2)
        return places

    def get_place_details(self, place_id, fields):
        endpoint_url = "https://maps.googleapis.com/maps/api/place/details/json"
        params = {
            'placeid': place_id,
            'fields': ",".join(fields),
            'key': self.apiKey
        }
        res = requests.get(endpoint_url, params = params)
        place_details =  json.loads(res.content)
        return place_details
if __name__ == '__main__':
    api = GooglePlaces("YOU_API_KEY")
    places = api.search_places_by_coordinate("19.368187, -99.276395", "15000", "restaurant") 

    fields = ['place_id', 'name', 'opening_hours', 'types',  'formatted_address', 'international_phone_number', 'website', 'rating', 'review', 'vicinity']
    for place in places:
        details = api.get_place_details(place['place_id'], fields)
        
        try:
            place_id = place['place_id']
        except KeyError:
            place_id = ""
        try:
            website = details['result']['website']
        except KeyError:
            website = ""

        try:
            name = details['result']['name']
        except KeyError:
            name = ""
        
        try:
            opening_hours = details['result']['opening_hours']
        except KeyError:
            opening_hours = ""

        try:
            types = details['result']['types']
        except KeyError:
            types = ""

        try:
            address = details['result']['formatted_address']
        except KeyError:
            address = ""

        try:
            phone_number = details['result']['international_phone_number']
        except KeyError:
            phone_number = ""

        try:
            rating = details['result']['rating']
        except KeyError:
            rating = ""

        try:
            reviews = details['result']['reviews']
        except KeyError:
            reviews = []
        
        try:
            vicinity = details['result']['vicinity']
        except KeyError:
            vicinity = ""
        

        print("===================PLACE===================")
        print("Name:", name)
        print("Website:", website)
        print("Address:", address)
        print("Phone Number", phone_number)
        data = [place_id,name, website,opening_hours, types, address, phone_number, rating, reviews,vicinity]
        final_data.append(data)
        print("==================REWIEVS==================")
        for review in reviews:
            author_name = review['author_name']
            rating = review['rating']
            text = review['text']
            time = review['relative_time_description']
            profile_photo = review['profile_photo_url']
            print("Author Name:", author_name)
            print("Rating:", rating)
            print("Text:", text)
            print("Time:", time)
            print("Profile photo:", profile_photo)
            print("-----------------------------------------")


In [ ]:
import pandas as pd, numpy as np
labels = ['place_id','Place Name','website', 'opening_hours','types', 'address', 'phone_number', 'rating', 'reviews', 'vicinity']

export_dataframe_1 = pd.DataFrame.from_records(final_data, columns=labels)
export_dataframe_1.to_csv('exampleExportGooglePlace.csv')

with open('exportjsonplace.json', 'w') as json_file:
    json.dump(places, json_file)
